# Demo de algoritmos para almacenar passwords

In [21]:
password = 'cbda'

In [22]:
ALPHABET = 'abcd'

## Probando las funciones hash clásicas: MD5, SHA1 y SHA256

In [23]:
from Crypto.Hash import MD5, SHA, SHA256

In [25]:
md5 = MD5.new()
md5.update(password.encode('utf-8'))
print(md5.hexdigest())

4abf4d24c623f9a3fa9ca35f68b9ce17


In [27]:
sha1 = SHA.new()
sha1.update(password.encode('utf-8'))
print(sha1.hexdigest())

270a5f78075011a3f9d3861b20e9dc5161df8851


In [28]:
sha256 = SHA256.new()
sha256.update(password.encode('utf-8'))
print(sha256.hexdigest())

b6d838d324733bba724b52e721620abcbde75183e449a42b3f4edfa974a3222b


Ahora, una función que me permita generar todas las passwords posibles (del largo del alfabeto) para poder hacer fuerza bruta, y una segunda que me permita hacer la fuerza bruta para cada algoritmo:

In [29]:
import itertools

def get_bruteforce_string_list():
    prod = itertools.product(ALPHABET, repeat = len(ALPHABET))
    return (("".join(map(str, a)) for a in prod))
def bruteforce_hash(hash_object, password_hash):
    for attempt in get_bruteforce_string_list():
        if hash_object.new(attempt.encode('utf-8')).hexdigest() == password_hash.hexdigest():
            print('PASSWORD FOUND: %s' % attempt)
            break
def bruteforce_plaintext():
    for attempt in get_bruteforce_string_list():
        if attempt == password:
            print('PASSWORD FOUND: %s' % attempt)
            break

#### Fuerza bruta para texto plano

In [30]:
timeit_plaintext = %%timeit -n1 -r1 -q -o bruteforce_plaintext()

PASSWORD FOUND: cbda


#### Fuerza bruta para MD5

In [31]:
timeit_md5 = %%timeit -n1 -r1 -q -o bruteforce_hash(MD5, md5)

PASSWORD FOUND: cbda


#### Fuerza bruta para SHA1

In [32]:
timeit_sha1 = %%timeit -n1 -r1 -q -o bruteforce_hash(SHA, sha1)

PASSWORD FOUND: cbda


#### Fuerza bruta para SHA256

In [33]:
timeit_sha256 = %%timeit -n1 -r1 -q -o bruteforce_hash(SHA256, sha256)

PASSWORD FOUND: cbda


## Ahora con una función KDF: BCrypt

In [34]:
import bcrypt

Generación de una sal. A la función `gensalt` se le puede dar un parámetro de costo. Por defecto es 12. Ver <https://en.wikipedia.org/wiki/Bcrypt>

In [35]:
salt = bcrypt.gensalt()
print(salt)

b'$2b$12$FxvzUYipOTC6O6EHreETHO'


Generación del KDF o "digest" respectivo:

In [37]:
kdf = bcrypt.hashpw(password.encode('utf-8'), salt)
print(kdf)

b'$2b$12$FxvzUYipOTC6O6EHreETHOdZKB40x0u0.SK87pLj7AarWPUFMx7jm'


#### Fuerza bruta para BCrypt

In [38]:
def bruteforce_bcrypt():
    for attempt in get_bruteforce_string_list():
        if bcrypt.hashpw(attempt.encode('utf-8'), kdf) == kdf:
            print('PASSWORD FOUND: %s' % attempt)
            break
timeit_kdf = %%timeit -n1 -r1 -q -o bruteforce_bcrypt()

PASSWORD FOUND: cbda


## Resultados

In [39]:
from ipy_table import make_table, apply_theme
results_data = [
    ['Tipo', 'Tiempo (s)'],
    ['Texto plano', timeit_plaintext.best],
    ['MD5', timeit_md5.best],
    ['SHA1', timeit_sha1.best],
    ['SHA256', timeit_sha256.best],
    ['KDF (BCrypt)', timeit_kdf.best],
]
make_table(results_data)
apply_theme('basic')

Tipo,Tiempo (s)
Texto plano,0.0005
MD5,0.0019
SHA1,0.0011
SHA256,0.0011
KDF (BCrypt),43.3847
